In [6]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [7]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from functools import singledispatch
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

from google.colab import drive

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [125]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [126]:
ls data

mens-shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/mens-shoes.csv', low_memory=False)

In [128]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [130]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [131]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [132]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [133]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [134]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [135]:
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

#  {'key': 'Gender', 'value': ['Men']}


df['features_parsed'] = df['features'].map(parse_features)

In [137]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [138]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [139]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [140]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [141]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_stock no.', 'feat_variant group id', 'feat_temple length',
       'feat_ean', 'feat_product #', 'feat_lens technology', 'feat_ground',
       'feat_gender', 'feat_necklace type', 'feat_chain length'],
      dtype='object', length=526)

In [142]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100   #  In how many% does this feature occur

0.0437636761487965

In [143]:
df.shape[0]

18280

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100 

In [145]:
{k:v for k, v in keys_stat.items() if v > 30}   #  feature that occurs in 30% of footwear as a parameter

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#  feature engineering
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

#  feature engineering for all columns
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [147]:
print(df[ df.brand  != df.feat_brand ].shape)
df[ df.brand  == df.feat_brand ].shape

(18228, 1002)


(52, 1002)

In [148]:
df[ df.brand  == df.feat_brand ][ [ 'brand', 'feat_brand']].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [149]:
df[ df.brand  != df.feat_brand ][ [ 'brand', 'feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [150]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )    #  normalization
df[ df.brand  != df.feat_brand ][ [ 'brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [151]:
print(df[ df.brand  != df.feat_brand ].shape)
df[ df.brand  == df.feat_brand ].shape

(9434, 1002)


(8846, 1002)

In [0]:
feats = ['']

In [153]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.31077322269092, 4.19251950868551)

In [166]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_certifications and listings',
 'feat_location - city/state',
 'feat_catalog',
 'feat_location - country',
 'feat_shoe category',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_msrp_cat',
 'feat_neckline_cat',
 'feat_outer material_cat',
 'feat_frame type_cat',
 'feat_power type_cat',
 'feat_instrument_cat',
 'feat_occasion_cat',
 'feat_lens tint_cat',
 'feat_diamond color_cat',
 'feat_colour code_cat',
 'feat_size/dimensions_cat',
 'feat_number of items_cat',
 'feat_article_cat',
 'feat_ring style_cat',
 'feat_assembled in country of origin_cat',
 'feat_terrain_cat',
 'feat_footwear type_cat',
 'feat_jewelry setting_cat',
 'feat_front style_cat',
 'feat_color mapping_cat',
 'feat_sock size_cat',
 'feat_style #_cat',
 'feat_shippin

In [199]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']
feats += [#
           'feat_special features_cat',
 'feat_clothing size_cat',
 'feat_origin of components_cat',
 'feat_gold karat_cat',
 'feat_sports team_cat',
 'feat_battery voltage_cat',
 'feat_mpn#_cat',
 'feat_материал_cat',
 'feat_band color_cat',
 'feat_fits model_cat',
 'feat_release_cat',
 'feat_polarized_cat',
 'feat_clothing size type_cat',
 'feat_is portable_cat',
 'feat_sub style_cat',
 'feat_autographed_cat',
 'feat_size_cat',
 'feat_subdial_cat',
 'feat_protects against_cat',
 'feat_stability_cat',
 'feat_transactionid_cat',
 'feat_manufacturer_part_number_cat',
 'feat_safety feature_cat',
 'feat_batteries required?_cat',
 'feat_lens width_cat',
 'feat_light weight_cat',
 'feat_year_cat',
 'feat_era_cat',
 'feat_age_cat',
 'feat_leather :_cat',
 'feat_construction_cat',
 'feat_bag size range_cat',
 'feat_fuel type_cat',
 'feat_colour_cat',
 'feat_compatible devices_cat',
 'feat_fabric content_cat',
 'feat_lined_cat',
 'feat_location - country_cat',
 'feat_finish_cat',
 'feat_hood_cat',
 'feat_shoe height_cat',
 'feat_main colour_cat',
 'feat_number of compartments_cat',
 'feat_spikes type_cat',
 'feat_band type_cat',
 'feat_retail price_cat',
 'feat_is recyclable_cat',
 'feat_looks best on these face shapes:_cat',
 'feat_manufacturer number_cat',
 'feat_shoe category_cat',
 'feat_dimensions_cat',
 'feat_expandable_cat',
 'feat_upper material_cat',
 'feat_jacket length_cat',
 'feat_to fit_cat',
 'feat_name_cat',
 'feat_insulation_cat',
 'feat_is dark sky-compliant_cat',
 'feat_use_cat',
 'feat_best sellers rank_cat',
 'feat_frame material:_cat',
 'feat_chest_cat',
 'feat_hunting_cat',
 'feat_foot arch_cat',
 'feat_hooded_cat',
 'feat_display_cat',
 'feat_product dimensions_cat',
 'feat_sock style_cat',
 'feat_carats_cat',
 'feat_contains batteries_cat',
 'feat_fits models_cat',
 'feat_ring size_cat',
 'feat_charger included_cat',
 'feat_material detail-1_cat',
 'feat_clothing category_cat',
 'feat_toe type_cat',
 'feat_impact resistant_cat',
 'feat_cm_cat',
 'feat_label_cat',
 'feat_place of origin_cat',
 'feat_elastic banded waist_cat',
 'feat_hat style_cat',
 'feat_number of batteries included_cat',
 'feat_clothing product type_cat',
 'feat_motion control_cat',
 'feat_conflict_cat',
 'feat_part number_cat',
 'feat_adidas_cat',
 'feat_removable hood_cat',
 'feat_product type_cat',
 'feat_heel height_cat',
 'feat_boot height_cat',
 'feat_metal color_cat',
 'feat_nike id_cat',
 'feat_ul safety listing_cat',
 'feat_alarm_cat',
 'feat_applicable_cat',
 'feat_parts_cat',
 'feat_lens color_cat',
 'feat_model number_cat',
 'feat_mirrored_cat',
 'feat_combined shipping_cat',
 'feat_item type_cat',
 'feat_nickel free_cat',
 'feat_fastener type_cat',
 'feat_sleeve length_cat',
 'feat_pant style_cat',
 'feat_fabric type_cat',
          #
 'feat_country of origin - components_cat',
 'feat_item weight_cat',
 'feat_chronograph_cat',
 'feat_sleeve style_cat',
 'feat_quantity in set_cat',
 'feat_auto_cat',
 'feat_used with equipment type_cat',
 'feat_type of sole_cat',
 'feat_count_cat',
 'feat_sku number_cat',
 'feat_face shape_cat',
 'feat_style name_cat',
 'feat_style code_cat',
 'feat_sports league_cat',
 'feat_free item_cat',
 'feat_band manufacturer_cat',
 'feat_watch shape_cat',
 'feat_bridge width_cat',
 'feat_reinforced knee_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_video game platform_cat',
 'feat_is orthopedic_cat',
 'feat_primary color_cat',
 'feat_net weight_cat',
 'feat_bracelet style_cat',
 'feat_heart rate monitor_cat',
 'feat_chain included_cat',
 'feat_shirt size_cat',
 'feat_bridge size:_cat',
 'feat_movement_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_style:_cat',
 'feat_manufacturer sku_cat',
 'feat_suitable for_cat',
 'feat_full product manual_cat',
 'feat_pattern_cat',
 'feat_characteristics_cat',
 'feat_leg_cat',
 'feat_dial color_cat',
 'feat_wind resistant_cat',
 'feat_audience_cat',
 'feat_is energy star-certified_cat',
 'feat_polarized lenses_cat',
 'feat_stone type_cat',
 'feat_chest size_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_is polarized_cat',
 'feat_power reserve_cat',
 'feat_case type_cat',
 'feat_number of heat settings_cat',
 'feat_water resistant_cat',
 'feat_frame color_cat',
 'feat_eyewear frame style_cat',
 'feat_weight_cat',
 'feat_thick, warm, cozy_cat',
 'feat_genre_cat',
 'feat_frame material_cat',
 'feat_part type_cat',
 'feat_high visibility (ansi compliant)_cat',
 'feat_szie_cat',
 'feat_shade_cat',
 'feat_features & fastening_cat',
 'feat_designed for_cat',
 'feat_uv protection_cat',
 'feat_leg opening:_cat',
 'feat_sizebridge_cat',
 'feat_fits over eyeglasses_cat',
 'feat_assembled product weight_cat',
 'feat_material composition_cat',
 'feat_manufacturer_cat',
 'feat_protection_cat',
 'feat_interior pockets_cat',
 'feat_color family_cat',
 'feat_has paper wood_cat',
 'feat_airport friendly_cat',
 'feat_charge time (minutes)_cat',
 'feat_casual & dress shoe style_cat',
 'feat_frame depth_cat',
 'feat_hardsided or softsided_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_casing_cat',
 'feat_insulated_cat',
 'feat_metal type_cat',
 'feat_lens material:_cat',
 'feat_digital camera_cat',
 'feat_eye size:_cat',
 'feat_productgroup_cat',
 'feat_athlete_cat',
 'feat_includes_cat',
 'feat_country/region of manufacture_cat',
 'feat_animal type_cat',
 'feat_watch style_cat',
 'feat_assembled product dimensions (l x w x h)_cat',
 'feat_multi pack indicator_cat',
 'feat_box_cat',
 'feat_display technology_cat',
 'feat_sku_cat',
 'feat_recommended location_cat',
 'feat_clothing type_cat',
 'feat_metal stamp_cat',
 'feat_number of pockets_cat',
 'feat_euro size_cat',
 'feat_watch band material_cat',
 'feat_age end_cat',
 'feat_sole_cat',
 'feat_fit to_cat',
 'feat_fabrication_cat',
 'feat_height_cat',
 'feat_fit:_cat',
 'feat_feature_cat',
 'feat_walmart no._cat',
 'feat_adjustable_cat',
 'feat_fits brands_cat',
 'feat_diameter_cat',
 'feat_wheeled_cat',
 'feat_what includes_cat',
 'feat_antiscratch lens coating_cat',
 'feat_capacity_cat',
 'feat_made in_cat',
 'feat_theme_cat',
 'feat_water-resistant_cat',
 'feat_bridge/temple size:_cat',
 'feat_fabric material_cat',
 'feat_is lined_cat',
 'feat_auto shut-off_cat',
 'feat_issued/ not-issued_cat',
 'feat_bezel_cat',
 'feat_frame style_cat',
 'feat_watch power source_cat',
 'feat_global composite sports type_cat',
 'feat_country of origin components:_cat',
 'feat_batteries included?_cat',
 'feat_main color_cat',
 'feat_is wheeled_cat',
 'feat_item width_cat',
 'feat_dial material_cat',
 'feat_bridge_cat',
 'feat_attachment_cat',
 'feat_age group_cat',
 'feat_country//region of manufacture_cat',
 'feat_waist_cat',
 'feat_polarised_cat',
 'feat_disclosures_cat',
 'feat_clasp type_cat',
 'feat_fastener_cat',
 'feat_stock no._cat',
 'feat_variant group id_cat',
 'feat_temple length_cat',
 'feat_ean_cat',
 'feat_product #_cat',
 'feat_lens technology_cat',
 'feat_ground_cat',
 'feat_necklace type_cat',
 'feat_chain length_cat']

#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.521345842037114, 4.217688088956118)

In [200]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2515 ± 0.0072,brand_cat
0.1073 ± 0.0046,feat_material_cat
0.0259 ± 0.0055,feat_adjustable_cat
0.0123 ± 0.0010,feat_weight_cat
0.0112 ± 0.0014,feat_brand_cat
0.0100 ± 0.0008,feat_fabric content_cat
0.0061 ± 0.0003,feat_movement_cat
0.0057 ± 0.0004,feat_shoe category_cat
0.0056 ± 0.0010,feat_color_cat
0.0042 ± 0.0003,feat_fabric material_cat


In [156]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.081962467368385, 4.181533154300189)

In [220]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat', 
         'feat_adjustable_cat', 'feat_weight_cat', 'feat_movement_cat'] 

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)
print(result)

(-57.051174714204556, 4.241658832387258)


In [221]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.051174714204556, 4.241658832387258)


Weight,Feature
0.2504 ± 0.0071,brand_cat
0.1071 ± 0.0058,feat_material_cat
0.0260 ± 0.0028,feat_adjustable_cat
0.0188 ± 0.0007,feat_brand_cat
0.0150 ± 0.0010,feat_movement_cat
0.0135 ± 0.0012,feat_weight_cat
0.0104 ± 0.0006,feat_gender_cat
0.0055 ± 0.0012,feat_color_cat


In [184]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2525 ± 0.0116,brand_cat
0.1076 ± 0.0098,feat_material_cat
0.0265 ± 0.0083,feat_adjustable_cat
0.0177 ± 0.0011,feat_brand_cat
0.0148 ± 0.0014,feat_movement_cat
0.0129 ± 0.0008,feat_weight_cat
0.0100 ± 0.0015,feat_gender_cat
0.0055 ± 0.0013,feat_color_cat
0.0012 ± 0.0001,feat_shape_cat


In [157]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [158]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.16110569712605, 4.23829320747004)

In [159]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2595 ± 0.0080,brand_cat
0.1099 ± 0.0134,feat_material_cat
0.0435 ± 0.0055,feat_gender_cat
0.0224 ± 0.0007,feat_brand_cat


In [160]:
df['brand'].value_counts(normalize=True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
lanbaosi        0.000055
eros            0.000055
surplus         0.000055
covona          0.000055
edward armah    0.000055
Name: brand, Length: 1732, dtype: float64

In [161]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [162]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
#  style 57.13,   sport 57.11,   null 57.16,  sport + style 57.04

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.15637248752613, 4.26154783363898)

In [163]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_gender_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [164]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'country/region of manufacture': 'china', 'mpn': '684832-014', 'color': 'black hyper turq'},
       {'sport': 'basketball', 'gender': 'boys', 'shoe category': 'boys shoes', 'brand': 'nike', 'color': 'tour yellow/hyper cobalt/copper flash', 'casual & dress shoe style': 'athletic', 'shoe width': 'medium'},
       {'style': 'n/a', 'lens technology': 'no,no,59,6', 'mpn': 'ev0674-420-202', 'gender': 'men', 'manufacturer': 'nike', 'shipping weight (in pounds)': '1.0', 'manufacturer part number': 'ev0674-420-202', 'brand': 'nike', 'age group': 'adult'},
       {'season': 'all-season', 'material': 'canvas', 'gender': 'men', 'shoe size': '10', 'size': '10', 'color': 'blue', 'model': '685150 447', 'manufacturer part number': '685150 447', 'brand': 'nike', 'age group': 'adult'},
       {}], dtype=object)

In [165]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
mens                   4
infant                 4
boys'                  3
youth                  2
women                  2
women ,�� unisex       2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [14]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [23]:
ls

ColabDriveGithub.ipynb  day4_meta.ipynb    LICENSE
data/                   day5.ipynb         matrix_one/
day4.ipynb              HelloGitHub.ipynb  README.md


In [0]:
!echo 'data' > .gitignore

In [0]:
!git add .gitignore

In [28]:
!git add matrix_one/day5.ipynb

ColabDriveGithub.ipynb  day4.ipynb       day5.ipynb
day3.ipynb              day4_meta.ipynb  HelloGitHub.ipynb
